<a href="https://colab.research.google.com/github/DeepRup/DRDOReID/blob/master/PersonReID_DUKEMTMC_ReID_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import os
import random
import keras
import numpy as np 
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from keras.layers import *
from keras.models import *
from keras.preprocessing.image import ImageDataGenerator

In [2]:
!unzip DukeMTMC-reID.zip

Streaming output truncated to the last 5000 lines.
  inflating: __MACOSX/DukeMTMC-reID/bounding_box_train/._4215_c8_f0042468.jpg  
  inflating: __MACOSX/DukeMTMC-reID/bounding_box_train/._4215_c8_f0042588.jpg  
  inflating: __MACOSX/DukeMTMC-reID/bounding_box_train/._4215_c8_f0042708.jpg  
  inflating: __MACOSX/DukeMTMC-reID/bounding_box_train/._4216_c6_f0063434.jpg  
  inflating: __MACOSX/DukeMTMC-reID/bounding_box_train/._4216_c6_f0063554.jpg  
  inflating: __MACOSX/DukeMTMC-reID/bounding_box_train/._4216_c6_f0063674.jpg  
  inflating: __MACOSX/DukeMTMC-reID/bounding_box_train/._4216_c6_f0063794.jpg  
  inflating: __MACOSX/DukeMTMC-reID/bounding_box_train/._4216_c6_f0063914.jpg  
  inflating: __MACOSX/DukeMTMC-reID/bounding_box_train/._4216_c6_f0064034.jpg  
  inflating: __MACOSX/DukeMTMC-reID/bounding_box_train/._4216_c6_f0064154.jpg  
  inflating: __MACOSX/DukeMTMC-reID/bounding_box_train/._4216_c7_f0068113.jpg  
  inflating: __MACOSX/DukeMTMC-reID/bounding_box_train/._4216_c7_f006

In [3]:
train_list=os.listdir("/content/DukeMTMC-reID/bounding_box_train")

l=len(train_list)
print(l)

traindf=pd.DataFrame(train_list)

train_label=[]
for n in train_list:
  arr=re.split(r'_',n)
  train_label.append(arr[0])

traindf['PersonID']=train_label
traindf.columns=['ImageName','PersonID']

print(traindf)

16522
                  ImageName PersonID
0      0454_c1_f0123223.jpg     0454
1      0860_c2_f0223238.jpg     0860
2      0282_c5_f0107630.jpg     0282
3      0778_c1_f0195315.jpg     0778
4      0465_c6_f0103476.jpg     0465
...                     ...      ...
16517  0059_c3_f0037468.jpg     0059
16518  5388_c6_f0140952.jpg     5388
16519  0628_c1_f0159780.jpg     0628
16520  0281_c5_f0107107.jpg     0281
16521  4195_c7_f0063282.jpg     4195

[16522 rows x 2 columns]


In [4]:
val_list=os.listdir("/content/DukeMTMC-reID/query")

l=len(val_list)
print(l)

valdf=pd.DataFrame(val_list)
val_label=[]

for n in val_list:
  arr=re.split(r'_',n)
  val_label.append(arr[0])

valdf['PersonID']=val_label
valdf.columns=['ImageName','PersonID']
print(valdf)

2228
                 ImageName PersonID
0     4072_c6_f0038383.jpg     4072
1     0369_c1_f0110100.jpg     0369
2     0500_c5_f0139946.jpg     0500
3     0264_c2_f0100382.jpg     0264
4     0136_c6_f0057115.jpg     0136
...                    ...      ...
2223  1297_c2_f0068237.jpg     1297
2224  1486_c4_f0069129.jpg     1486
2225  0292_c6_f0087135.jpg     0292
2226  1598_c3_f0089953.jpg     1598
2227  0200_c2_f0089080.jpg     0200

[2228 rows x 2 columns]


In [5]:
base_model=tf.keras.applications.ResNet50(input_shape=(224,224,3),
                                          weights='imagenet',
                                          include_top=False)
base_model.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__________________________

In [6]:
 for i, layer in enumerate(base_model.layers):
   print(i,layer.name)

0 input_1
1 conv1_pad
2 conv1_conv
3 conv1_bn
4 conv1_relu
5 pool1_pad
6 pool1_pool
7 conv2_block1_1_conv
8 conv2_block1_1_bn
9 conv2_block1_1_relu
10 conv2_block1_2_conv
11 conv2_block1_2_bn
12 conv2_block1_2_relu
13 conv2_block1_0_conv
14 conv2_block1_3_conv
15 conv2_block1_0_bn
16 conv2_block1_3_bn
17 conv2_block1_add
18 conv2_block1_out
19 conv2_block2_1_conv
20 conv2_block2_1_bn
21 conv2_block2_1_relu
22 conv2_block2_2_conv
23 conv2_block2_2_bn
24 conv2_block2_2_relu
25 conv2_block2_3_conv
26 conv2_block2_3_bn
27 conv2_block2_add
28 conv2_block2_out
29 conv2_block3_1_conv
30 conv2_block3_1_bn
31 conv2_block3_1_relu
32 conv2_block3_2_conv
33 conv2_block3_2_bn
34 conv2_block3_2_relu
35 conv2_block3_3_conv
36 conv2_block3_3_bn
37 conv2_block3_add
38 conv2_block3_out
39 conv3_block1_1_conv
40 conv3_block1_1_bn
41 conv3_block1_1_relu
42 conv3_block1_2_conv
43 conv3_block1_2_bn
44 conv3_block1_2_relu
45 conv3_block1_0_conv
46 conv3_block1_3_conv
47 conv3_block1_0_bn
48 conv3_block1_3_bn

In [7]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Flatten()(x)
x=Dense(1024,activation='relu')(x)
x=Dense(256,kernel_regularizer=tf.keras.regularizers.L2(0.001),
        activation='relu')(x)
x=Dense(1024,activation='relu')(x)
preds=Dense(702,activation='softmax')(x)

In [8]:
model=Model(inputs=base_model.input,outputs=preds)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [9]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_1
1 conv1_pad
2 conv1_conv
3 conv1_bn
4 conv1_relu
5 pool1_pad
6 pool1_pool
7 conv2_block1_1_conv
8 conv2_block1_1_bn
9 conv2_block1_1_relu
10 conv2_block1_2_conv
11 conv2_block1_2_bn
12 conv2_block1_2_relu
13 conv2_block1_0_conv
14 conv2_block1_3_conv
15 conv2_block1_0_bn
16 conv2_block1_3_bn
17 conv2_block1_add
18 conv2_block1_out
19 conv2_block2_1_conv
20 conv2_block2_1_bn
21 conv2_block2_1_relu
22 conv2_block2_2_conv
23 conv2_block2_2_bn
24 conv2_block2_2_relu
25 conv2_block2_3_conv
26 conv2_block2_3_bn
27 conv2_block2_add
28 conv2_block2_out
29 conv2_block3_1_conv
30 conv2_block3_1_bn
31 conv2_block3_1_relu
32 conv2_block3_2_conv
33 conv2_block3_2_bn
34 conv2_block3_2_relu
35 conv2_block3_3_conv
36 conv2_block3_3_bn
37 conv2_block3_add
38 conv2_block3_out
39 conv3_block1_1_conv
40 conv3_block1_1_bn
41 conv3_block1_1_relu
42 conv3_block1_2_conv
43 conv3_block1_2_bn
44 conv3_block1_2_relu
45 conv3_block1_0_conv
46 conv3_block1_3_conv
47 conv3_block1_0_bn
48 conv3_block1_3_bn

In [10]:
train_datagen=ImageDataGenerator(preprocessing_function=
                                 tf.keras.applications.resnet50
                                 .preprocess_input)
val_datagen=ImageDataGenerator(preprocessing_function=
                               tf.keras.applications.resnet50
                               .preprocess_input)

In [17]:
train_generator=train_datagen.flow_from_dataframe(traindf,
                                                 '/content/DukeMTMC-reID/bounding_box_train',
                                                  x_col='ImageName',
                                                  y_col='PersonID',
                                                  target_size=(224,224),
                                                  color_mode='rgb',
                                                  batch_size=32,
                                                  class_mode='categorical')

val_generator=val_datagen.flow_from_dataframe(valdf,
                                              '/content/DukeMTMC-reID/query',
                                              x_col='ImageName',
                                              y_col='PersonID',
                                              target_size=(224,224),
                                              color_mode='rgb',
                                              batch_size=32,
                                              class_mode='categorical')

Found 16522 validated image filenames belonging to 702 classes.
Found 2228 validated image filenames belonging to 702 classes.


In [18]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [19]:
history=model.fit(train_generator,epochs=21,validation_data=val_generator)

Epoch 1/21
517/517 [==============================] - 185s 358ms/step - loss: 0.3912 - accuracy: 0.8940 - val_loss: 26.8277 - val_accuracy: 0.0036
Epoch 2/21
517/517 [==============================] - 184s 356ms/step - loss: 0.3288 - accuracy: 0.9132 - val_loss: 27.8996 - val_accuracy: 0.0031
Epoch 3/21
517/517 [==============================] - 184s 355ms/step - loss: 0.2865 - accuracy: 0.9279 - val_loss: 27.4323 - val_accuracy: 0.0027
Epoch 4/21
517/517 [==============================] - 184s 356ms/step - loss: 0.2898 - accuracy: 0.9244 - val_loss: 29.5189 - val_accuracy: 0.0027
Epoch 5/21
517/517 [==============================] - 184s 356ms/step - loss: 0.2481 - accuracy: 0.9376 - val_loss: 31.5628 - val_accuracy: 0.0058
Epoch 6/21
517/517 [==============================] - 184s 355ms/step - loss: 0.2411 - accuracy: 0.9408 - val_loss: 31.3481 - val_accuracy: 0.0031
Epoch 7/21
517/517 [==============================] - 184s 356ms/step - loss: 0.2452 - accuracy: 0.9386 - val_loss: 26